In [1]:
import pandas as pd
import numpy as np
import pyodbc as db
import os
import glob
from glob import glob
import re
import luigi
from operator import itemgetter
from persiantools import characters
from persiantools.jdatetime import JalaliDate
import persiantools
import datetime
from re import search
import itertools
from statistics import mode
import datetime as dt
from joblib import Parallel, delayed
import chardet
import requests
from bs4 import BeautifulSoup
import json
import time
import io
import jdatetime
import TSETMCScraping
import ar_to_fa
import clean_cols
import str_to_date
import str_to_jdate
import str_to_time
import cleaning_path
from luigi.mock import MockTarget
import stock_information_cleaning

In [54]:
class TSETMC_Sraping(luigi.Task):
    read_path= luigi.Parameter(default=None)
    dirr_path= luigi.Parameter(default=None)
    def requiers(self):
        return None
    def output(self):
        return luigi.LocalTarget('log.txt')
    # def output(self):
    #     return MockTarget("first task", mirror_on_stderr=True)
    def run(self):
        Data_scraping= input('wich data do you want to scrap: (trade\information\shareholders) ')
        if Data_scraping =='information':
            if os.path.exists(self.dirr_path):
                pass
            else:
                scrap_links= pd.read_excel(self.read_path)
                df= TSETMCScraping.stock_information(scrap_links)
                # df= cleaning_path.column_clustering(df)
                # df= stock_information_cleaning.string_clean(df)
                # df= stock_information_cleaning.price_clean(df)
                # df= stock_information_cleaning.code_clean(df)
                f= self.output().path.open('w')
                df.to_csv(self.read_path.split('\\')[:4]+'\\output.csv', encoding='utf-8-sig',header=True)
        if Data_scraping == 'trade':
            if os.path.exists(self.dirr_path):
                pass
            else:
                scrap_links= pd.read_excel(self.read_path)
                df= TSETMCScraping.trade_history(scrap_links)
                # df= cleaning_path.clean_cols(df)
                # df= cleaning_path.ar_to_fa(df)
                # df= cleaning_path.str_to_date(df)
                # df= cleaning_path.date_to_jdate(df)
                df.to_csv(self.output().path, index=False, encoding='utf-8-sig')
        if Data_scraping == 'shareholders':
            if os.path.exists(self.dirr_path):
                pass
            else:
                scrap_links= pd.read_excel(self.read_path)
                df= TSETMCScraping.shareholders(scrap_links)
                df.to_csv(self.output().path, index=False, encoding='utf-8-sig')

In [19]:
class Insert_to_DB (luigi.Task):
    read_path= luigi.Parameter(default=None)
    def requiers(self):
        return TSETMC_Sraping ()
    def run (self):
        input_1= input('Server name: ')
        input_2= input('Data base name: ')
        input_3=input('User name: ')
        input_4= input('Password: ')
        server = input_1
        database= input_2
        username= input_3
        password= input_4
        if input_2=='information':
            cnxn= db.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID'+username+';PWD='+password)
            cursor= cnxn.cursor()
            df= self.input()
            for index, row in df.iterrows():
                cursor.execute('''
                            INSERT INTO profit ('code_stock_12','code_stock_5','company_name_English','code_company_4','company_name','stock_name','stock_persian_30',
                            'code_company_12','market','code_tablo','code_sector','name_sector','code_subsector','name_subsector',
                            'stock_name_2','trade_purpose','CEO','address','phone_number','FAX','offic_address',
                            'shareholder_affairs_address','website','email','auditor','capital','year','financial_manager',
                            'shenaseh_meli','stock_code_TSETMC')
                            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                            ''',
                            row.loc['code_stock_12'], 
                            row.loc['code_stock_5'],
                            row.loc['company_name_English'],
                            row.loc['code_company_4'],
                            row.loc['company_name'],
                            row.loc['stock_name'],
                            row.loc['stock_persian_30'],
                            row.loc['code_company_12'],
                            row.loc['market'],
                            row.loc['code_tablo'],
                            row.loc['code_sector'],
                            row.loc['name_sector'],
                            row.loc['code_subsector'],
                            row.loc['name_subsector'],
                            row.loc['stock_name_2'],
                            row.loc['trade_purpose'],
                            row.loc['CEO'],
                            row.loc['address'],
                            row.loc['phone_number'],
                            row.loc['FAX'],
                            row.loc['offic_address'],
                            row.loc['shareholder_affairs_address'],
                            row.loc['website'],
                            row.loc['email'],
                            row.loc['auditor'],
                            row.loc['capital'],
                            row.loc['year'],
                            row.loc['financial_manager'],
                            row.loc['shenaseh_meli'],
                            row.loc['stock_code_TSETMC']
                            )
            cnxn.commit()
            cursor.close()
        if input_2=='trade':
            cnxn= db.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID'+username+';PWD='+password)
            cursor= cnxn.cursor()
            df= self.input()
            for index, row in df.iterrows():
                cursor.execute('''
                            INSERT INTO profit ('insCode','dEven','nTran','hEven','qTitTran','pTran','qTitNgJ','iSensVarP','pPhSeaCotJ','pPbSeaCotJ',
                            'iAnuTran','xqVarPJDrPRf','canceledCompony','shenaseh_meli','Date')
                            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                            ''',
                            row.loc['insCode'], 
                            row.loc['dEven'],
                            row.loc['nTran'],
                            row.loc['hEven'],
                            row.loc['qTitTran'],
                            row.loc['pTran'],
                            row.loc['qTitNgJ'],
                            row.loc['iSensVarP'],
                            row.loc['pPhSeaCotJ'],
                            row.loc['pPbSeaCotJ'],
                            row.loc['iAnuTran'],
                            row.loc['xqVarPJDrPRf'],
                            row.loc['canceledCompony'],
                            row.loc['shenaseh_meli'],
                            row.loc['Date']
                            )
            cnxn.commit()
            cursor.close()
        if input_3=='shareholders':
            cnxn= db.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID'+username+';PWD='+password)
            cursor= cnxn.cursor()
            df= self.input()
            for index, row in df.iterrows():
                cursor.execute('''
                            INSERT INTO profit ('Shareholder','Share','Percent','Change','Compony','shenaseh_meli','shareholder_id')
                            VALUES (?,?,?,?,?,?,?)
                            ''',
                            row.loc['Shareholder'], 
                            row.loc['Share'],
                            row.loc['Percent'],
                            row.loc['Change'],
                            row.loc['Compony'],
                            row.loc['shenaseh_meli'],
                            row.loc['shareholder_id']
                            )
            cnxn.commit()
            cursor.close()

Runnnig Luigi

In [55]:
if __name__=='__main__':
    luigi.build([TSETMC_Sraping(read_path="C:\\Users\\ah.roudsaz\\Desktop\\test.xlsx",
                dirr_path="C:\\Users\\ah.roudsaz\\Desktop\\output.csv")],local_scheduler=True)

DEBUG: Checking if TSETMC_Sraping(read_path=C:\Users\ah.roudsaz\Desktop\test.xlsx, dirr_path=C:\Users\ah.roudsaz\Desktop\output.csv) is complete
INFO: Informed scheduler that task   TSETMC_Sraping_C__Users_ah_roud_C__Users_ah_roud_6f6acf47fd   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 53764] Worker Worker(salt=137099676, workers=1, host=POOLBANK-RDS2, username=ah.roudsaz, pid=53764) running   TSETMC_Sraping(read_path=C:\Users\ah.roudsaz\Desktop\test.xlsx, dirr_path=C:\Users\ah.roudsaz\Desktop\output.csv)
ERROR: [pid 53764] Worker Worker(salt=137099676, workers=1, host=POOLBANK-RDS2, username=ah.roudsaz, pid=53764) failed    TSETMC_Sraping(read_path=C:\Users\ah.roudsaz\Desktop\test.xlsx, dirr_path=C:\Users\ah.roudsaz\Desktop\output.csv)
Traceback (most recent call last):
  File "C:\Users\ah.roudsaz\AppData\Local\Programs\Python\Python310\lib\site-packages\luigi\worker.py",